
## Neural Machine Translation Project


In [ ]:
import collections

import helper
import numpy as np
import pandas as pd

!pip install indic-nlp-library
import indicnlp
#from indicnlp.tokenize import indic_tokenize
from indicnlp.normalize.indic_normalize import IndicNormalizerFactory

import tensorflow
from tensorflow.keras.optimizers import Adam

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, LSTM, Dropout
from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam

from keras.losses import sparse_categorical_crossentropy

     |████████████████████████████████| 40 kB 5.0 MB/s 
     |████████████████████████████████| 2.8 MB 55.9 MB/s 


### Verify access to the GPU


In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8064935868162142763
xla_global_id: -1
]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


.
### Load Data


In [ ]:

#kannada_sentences = np.loadtxt('kn_tr.txt',dtype=str)
#kannada_sentences = pd.read_csv('kn_tr.txt', sep = '\n')
import string
import re
input_lang = open('/content/drive/MyDrive/Datasets/Kannada-Sanskrit/train/kn_tr.txt')
input_language_sent = []
for x in input_lang:
  x = re.sub(r'\n',' ',x)
  input_language_sent.append(re.sub(r'[^\w\s]', '', x))


#sanskrit_sentences = pd.read_csv('sn_train.txt')
target_lang=open('/content/drive/MyDrive/Datasets/Kannada-Sanskrit/train/sn_train.txt')
target_language_sent=[]
for x in target_lang:
  x = re.sub(r'\n', ' ', x)
  target_language_sent.append(re.sub(r'[^\w\s]', '', x))

print('Dataset Loaded')

Dataset Loaded


In [ ]:
input_language_words_counter = collections.Counter([word for sentence in input_language_sent for word in sentence.split()])
target_language_words_counter = collections.Counter([word for sentence in target_language_sent for word in sentence.split()])

print('{} Kannada words.'.format(len([word for sentence in input_language_sent for word in sentence.split()])))
print('{} unique kannada words.'.format(len(input_language_words_counter)))
print('10 Most common words in the Kannada dataset:')
print('"' + '" "'.join(list(zip(*input_language_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Sanskrit words.'.format(len([word for sentence in target_language_sent for word in sentence.split()])))
print('{} unique Sanskrit words.'.format(len(target_language_words_counter)))
print('10 Most common words in the Sanskrit dataset:')
print('"' + '" "'.join(list(zip(*target_language_words_counter.most_common(10)))[0]) + '"')

35046 Kannada words.
6395 unique kannada words.
10 Most common words in the Kannada dataset:
"ಮಡತತನ" "ನವ" "ಹರಗ" "ಈ" "ಇದ" "ಅವನನನ" "ಉತತಮ" "ನನ" "ಅವನ" "ನಡಯತತದದನ"

36996 Sanskrit words.
5579 unique Sanskrit words.
10 Most common words in the Sanskrit dataset:
"करत" "असत" "न" "बह" "समयक" "तम" "भवत" "वसतरम" "इत" "मम"


In [ ]:
from indicnlp.tokenize import indic_tokenize
indic_string='सुनो, कुछ आवाज़ आ रही है। फोन?'

print('Input String: {}'.format(indic_string))
print('Tokens: ')
for t in indic_tokenize.trivial_tokenize(indic_string):
    print(t)

Input String: सुनो, कुछ आवाज़ आ रही है। फोन?
Tokens: 
सुनो
,
कुछ
आवाज़
आ
रही
है
।
फोन
?



## Preprocess
Converting the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [ ]:
from indicnlp.tokenize import indic_tokenize
from collections import defaultdict

#The Counter variable stores the global position of each word based on its first occurrance
#next_i and id generator is used to generate ids for each unique word
'''
COUNTER = defaultdict(lambda : 0)
def next_i(counter_id=0):
    """Return a new ID"""
    global COUNTER
    COUNTER[counter_id] += 1
    return COUNTER[counter_id]

id_generator = defaultdict(lambda : next_i(0))
'''
#Running the below code gives maximum recursion depth reached error as my_tokenizer is recursed repeatedly (NEED TO CHECK)

#SOLVING : The copy paste of bottom code was generating the problem , causing infinite recursion (changed the my_tokenizer to get an array of sentences and return a)

#my_tokenizer tokenizes the text and returns 1.list where each sentence is converted into a list of integers where each word is mapped to unique integer
# 2.the dictionary/vocabulary of the language
def my_tokenizer(text):
    COUNTER = defaultdict(lambda : 0)
    def next_i(counter_id=0):
        """Return a new ID"""
        #Not using the global counter since it keeps incrementing cout each time the cell is run or the function is called since it was gloabl to the function
        #global COUNTER
        COUNTER[counter_id] += 1
        return COUNTER[counter_id]

    id_generator = defaultdict(lambda : next_i(0))
    token_list=[]
    token_id = []
    for sentence in text:
        temp = []
        for t in indic_tokenize.trivial_tokenize(sentence):
            temp.append(t)
            my_id=[id_generator[w] for w in temp]
        token_list.append(temp)
        token_id.append(my_id)

    return token_id, dict(id_generator)


'''def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
#tests.test_tokenize(tokenize)
'''
#def tokenize(x):



# Tokenize Example output
#text_sentences = ['ನಿಮ್ಮ ನಿವಾಸ ಎಲ್ಲಿದೆ','ನಿವಾಸ ಎಲ್ಲಿದೆ','ದಿಲೀಪನು ಪ್ರಾಣಿಗಳನ್ನು ಪ್ರೀತಿಸುತ್ತಾನೆ']

#rand, my_ids=my_tokenizer(text_sentences)
#print(rand)
#print(my_ids)
#text_tokenized, text_tokenizer = tokenize(text_sentences)
#print(text_tokenizer.word_index)
#print()
#for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
#    print('Sequence {} in x'.format(sample_i + 1))
#    print('  Input:  {}'.format(sent))
#     print('  Output: {}'.format(token_sent))

'def tokenize(x):\n    """\n    Tokenize x\n    :param x: List of sentences/strings to be tokenized\n    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)\n    """\n    # TODO: Implement\n    x_tk = Tokenizer()\n    x_tk\n    x_tk.fit_on_texts(x)\n    return x_tk.texts_to_sequences(x), x_tk\n#tests.test_tokenize(tokenize)\n'

### Padding
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.


In [ ]:
#padding with 0s at the end of the sentence to make size of each vector equal to size of longest sentence
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length=max([len(sentence) for sentence in x])
        print(length)

    return pad_sequences(x, maxlen=length, padding ='post')
#t=tests.test_pad(pad)
#print(t)

'''
# Pad Tokenized output
test_pad = pad(tokens)
print(test_pad)
for sample_i, (token_sent, pad_sent) in enumerate(zip(tokens, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))
'''

"\n# Pad Tokenized output\ntest_pad = pad(tokens)\nprint(test_pad)\nfor sample_i, (token_sent, pad_sent) in enumerate(zip(tokens, test_pad)):\n    print('Sequence {} in x'.format(sample_i + 1))\n    print('  Input:  {}'.format(np.array(token_sent)))\n    print('  Output: {}'.format(pad_sent))\n"

### Preprocess Pipeline
.

In [ ]:
#Removing x_tk and y_tk as word counter already present
#preprocesses the data and returns the list of vector representations and dictionaries of both the input and target language
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, input_language_dict =my_tokenizer(x)
    preprocess_y, target_language_dict = my_tokenizer(y)
    print(input_language_dict)
    print(target_language_dict)
    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y,input_language_dict,target_language_dict

#preproc_input_language_sentences= vector representations(matrix) of input language
#preproc_target_language_sent= vector representations of target language
#input_language_tokenizer= input language dictionary
#target_language_tokenizer=target language dictionary
preproc_input_language_sentences, preproc_target_language_sent,input_language_tokenizer,target_language_tokenizer = preprocess(input_language_sent, target_language_sent)


max_input_language_sequence_length = preproc_input_language_sentences.shape[1]
max_target_language_sequence_length = preproc_target_language_sent.shape[1]
input_language_vocab_size = len(input_language_words_counter)
target_language_vocab_size = len(target_language_words_counter)
max_length=max(max_input_language_sequence_length,max_target_language_sequence_length)
print(max_length)
print('Data Preprocessed')
print("Max Input Language sentence length:", max_input_language_sequence_length)
print("Max Target Language sentence length:", max_target_language_sequence_length)
print("Input Language vocabulary size:", input_language_vocab_size)
print("Target Language vocabulary size:", target_language_vocab_size)

{'ಅನಜನ': 1, 'ವಹನವನನ': 2, 'ಚಲಯಸತತದದನ': 3, 'ವಜಯನ': 4, 'ತಬ': 5, 'ಬಡವನಗದದನ': 6, 'ಗರಶನ': 7, 'ಸಲ': 8, 'ಪಡಯತತನ': 9, 'ನಜಡನ': 10, 'ಚಹವನನ': 11, 'ಮಡತತದದನ': 12, 'ಅಮತನ': 13, 'ಕಳಕದ': 14, 'ಉಡಪನನ': 15, 'ಧರಸದದನ': 16, 'ಅಧಶನ': 17, 'ಹರಗ': 18, 'ಹಗವ': 19, 'ಅಗತಯವದ': 20, 'ನವಲಲರ': 21, 'ಯವಗ': 22, 'ಪರರಭಸದದರ': 23, 'ಇದದ': 24, 'ಸವತತರ': 25, 'ಸಫಟವರ': 26, 'ಆಗದ': 27, 'ರಜನ': 28, 'ಅಸಭಯವಗ': 29, 'ವರತಸತತದದನ': 30, 'ತಮಮ': 31, 'ಸವಕಯ': 32, 'ಸರಕಷತ': 33, 'ವವರಸಕತವನನ': 34, 'ರಚಸ': 35, 'ಸದರಯದ': 36, 'ರಜನಗದದನ': 37, 'ಯಗರಜ': 38, 'ಹರಹರ': 39, 'ಅಲದಡವ': 40, 'ಸತ': 41, 'ಅತಥಗಳ': 42, 'ಬದದದರ': 43, 'ಈ': 44, 'ಪಜಮ': 45, 'ಉದದವಗದ': 46, 'ಅರಜನನ': 47, 'ಊಟವನನ': 48, 'ಖಲ': 49, 'ಸಭವಸದ': 50, 'ಎಲಲವ': 51, 'ಯರಬಬರ': 52, 'ಮನವಯನನ': 53, 'ತರಸಕರಸವದ': 54, 'ಒಳಳಯದಲಲ': 55, 'ಶಲಯ': 56, 'ವದಯರಥಯಬಬ': 57, 'ಹತತನ': 58, 'ವರಷಕ': 59, 'ಪರಕಷಯಲಲ': 60, 'ರಜಯದಲಲ': 61, 'ಪರಥಮ': 62, 'ಸಥನ': 63, 'ಪಡದದದನ': 64, 'ಜಗದಶನ': 65, 'ಏನ': 66, 'ಸವಯಪರರತವಗ': 67, 'ಸಭವಸವದಲಲ': 68, 'ಸರವಶನ': 69, 'ಹಸಬಟಟಯನನ': 70, 'ತಟಟದದನ': 71, 'ಇದ': 72, 'ಹಚಚ': 73, 'ಓದಲ': 74, 'ಸಧಯವಗಲಲಲ': 75, 'ಸತಯ': 76, 'ನಡಯತತದದನ': 77, 'ನಲಕಠನ': 78, 

## Models
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN


### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want. The function `logits_to_text` will bridge the gab between the logits from the neural network to the required translation.  

In [ ]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    #changed from tokenizer.word_index.items to tokenizer.items() coz tokenizer here is a dictionary and not keras tokenizer object
    index_to_words = {id: word for word, id in tokenizer.items()}
    index_to_words[0] = ''
    #print("logits:",logits)

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: Simple RNN

In [ ]:
def simple_model(input_shape, output_sequence_length, input_language_vocab_size, target_language_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param input_language_vocab_size: Number of unique input language words in the dataset
    :param target_language_vocab_size: Number of unique target language words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate=2e-3
    input_seq= Input(input_shape[1:])
    rnn=GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(target_language_vocab_size+1))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    model.summary()
    return model
#tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_input_language_sentences, max_length)
tmp_x = tmp_x.reshape((-1, max_length, 1))
preproc_target_language_sent = pad(preproc_target_language_sent, max_length)


# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_length,
    input_language_vocab_size,
    target_language_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_target_language_sent, batch_size=1024, epochs=4, validation_split=0.2)

# Print prediction(s)
#print(tmp_x[:1])[0])
#print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_language_tokenizer))

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 23, 1)]           0         
                                                                 
 gru_5 (GRU)                 (None, 23, 64)            12864     
                                                                 
 time_distributed_5 (TimeDis  (None, 23, 5580)         362700    
 tributed)                                                       
                                                                 
 activation_5 (Activation)   (None, 23, 5580)          0         
                                                                 
Total params: 375,564
Trainable params: 375,564
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
8/8 [==============================] - 27s 3s/step - loss: 8.5248 - accuracy: 0.6334 - val_loss: 8.3192 - val

In [ ]:
# Print prediction(s)
#print(tmp_x[:,:,0])
#print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], target_language_tokenizer))

temp = simple_rnn_model.predict(tmp_x[1:2])
#print(temp.shape)
print(logits_to_text(temp[0], target_language_tokenizer))

In [ ]:
np.argmax(temp)

array([[ 4,  2,  2, ..., 22, 22, 22]])

In [ ]:
np.argmax(temp[0,:,:],1)

array([21, 21, 21,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])

In [ ]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:2])[1], tokenizer))


In [ ]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:3])[2], tokenizer))

In [ ]:
print(logits_to_text(simple_rnn_model.predict(tmp_x[:3])[2], tokenizer))

### Model 2: Embedding
 RNN model using embedding.

In [ ]:

def embed_model(input_shape, output_sequence_length, input_vocab_size, target_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param input_vocab_size: Number of unique target language words in the dataset
    :param target_vocab_size: Number of unique target language words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 256
    learning_rate = 2e-3


    rnn = GRU(batch, return_sequences=True)
    print(input_shape[1])
    embedding=Embedding(input_vocab_size, batch, input_length=input_shape[1])
    logits = TimeDistributed(Dense(target_vocab_size, activation='softmax'))

    model = Sequential()
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])


    # TODO: Implement
    return model
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_input_sentences, max_input_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_san_sent.shape[-2]))
print(tmp_x.shape)


# TODO: Train the neural network
model_with_embdding= embed_model(tmp_x.shape, max_target_sequence_length, input_vocab_size, target_vocab_size)
model_with_embdding.fit(tmp_x, preproc_san_sent, batch_size=1024, epochs=10, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(model_with_embdding.predict(tmp_x[:1])[0], tokenizer))

In [ ]:
print(logits_to_text(model_with_embdding.predict(tmp_x[:2])[1], tokenizer))

In [ ]:
print("batch 256: "+logits_to_text(model_with_embdding.predict(tmp_x[:2])[1], tokenizer))

In [ ]:
print(logits_to_text(model_with_embdding.predict(tmp_x[:3])[2], tokenizer))

In [ ]:
print("batch 256: "+logits_to_text(model_with_embdding.predict(tmp_x[:3])[2], tokenizer))

In [ ]:
print(logits_to_text(model_with_embdding.predict(tmp_x[:2])[1], tokenizer))

In [ ]:
print(model_with_embdding.summary())

### Model 3: Bidirectional RNNs

In [ ]:
def bd_model(input_shape, output_sequence_length, input_vocab_size, target_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param input_vocab_size: Number of unique input language words in the dataset
    :param target_vocab_size: Number of unique target language words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 256
    learning_rate = 1e-3

    rnn = GRU(batch, return_sequences=True, dropout=0.001)
    bidirectional=Bidirectional(rnn, input_shape = input_shape[1:])
    logits = TimeDistributed(Dense(target_vocab_size, activation='softmax'))

    model = Sequential()
    model.add(bidirectional)
    model.add(logits)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model
tests.test_bd_model(bd_model)
#print('tested')


# TODO: Reshape the input
#print(preproc_san_sent.shape[1])
tmp_x = pad(preproc_input_sentences, preproc_san_sent.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_san_sent.shape[-2], 1))


# TODO: Train the neural network
bidirectional_model= bd_model(tmp_x.shape, preproc_san_sent.shape[1], input_vocab_size, target_vocab_size)
bidirectional_model.fit(tmp_x, preproc_san_sent, batch_size=512, epochs=10, validation_split=0.2)



In [ ]:

print("batch 256, 10 epochs accuracy 0.70: "+logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256, 15 epochs accuracy 0.7206 : "+logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256, dropout 0.0, 25 epochs accuracy 0.8319: "+logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256, LSTM, learningrate 1e-3, dropout 0.01, 10 epochs accuracy batchsize 512 0.7846: "+logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256, LSTM, learningrate 1e-3, dropout 0, 20 epochs accuracy batchsize 512 0.8034: "+logits_to_text(bidirectional_model.predict(tmp_x[:1])[0], target_tokenizer))

### Model 4: Encoder-Decoder (OPTIONAL)
 The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.


In [ ]:
def encdec_model(input_shape, output_sequence_length, input_vocab_size, target_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param input_vocab_size: Number of unique input language words in the dataset
    :param target_vocab_size: Number of unique target language words in the dataset
    :return: Keras model built, but not trained
    """
    batch = 256
    learning_rate = 3e-3

    rnn = LSTM(batch, return_sequences=True, dropout=0.01)
    repeatVector=RepeatVector(output_sequence_length)
    logits = TimeDistributed(Dense(target_vocab_size, activation='softmax'))

    model = Sequential()
    model.add(LSTM(batch, input_shape = input_shape[1:]))
    model.add(repeatVector)
    model.add(rnn)
    model.add(logits)
    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    return model
# OPTIONAL: Implement
tests.test_encdec_model(encdec_model)
print('tested')


# TODO: Reshape the input

tmp_x = pad(preproc_input_sentences, preproc_san_sent.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_san_sent.shape[-2], 1))


# TODO: Train the neural network
encdec_model= encdec_model(tmp_x.shape, preproc_san_sent.shape[1], input_vocab_size, target_vocab_size)
encdec_model.fit(tmp_x, preproc_san_sent, batch_size=1024, epochs=20, validation_split=0.2)



In [ ]:
print("batch 256,  learningrate 1e-2, 20 epochs accuracy 0.6787: "+logits_to_text(encdec_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256,  learningrate 3e-3, 30 epochs accuracy 0.8119: "+logits_to_text(encdec_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256,  learningrate 3e-3, 50 epochs accuracy 0.8461: "+logits_to_text(encdec_model.predict(tmp_x[:1])[0], target_tokenizer))

In [ ]:
print("batch 256,  LSTM, learningrate 3e-3, 20 epochs, batch size 512, accuracy 0.7114: "+logits_to_text(encdec_model.predict(tmp_x[:1])[0], target_tokenizer))

### Model 5: Bidirectional RNN with embeddings


In [ ]:
def model_final(input_shape, output_sequence_length, input_vocab_size, target_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param input_vocab_size: Number of unique input langauge words in the dataset
    :param target_vocab_size: Number of unique target language words in the dataset
    :return: Keras model built, but not trained
    """
    units = 256
    learning_rate = 6e-3
    embedding=Embedding(input_vocab_size, units, input_length=input_shape[1])
    rnn = LSTM(units)
    rnn2 = LSTM(units, return_sequences=True)
    #print(input_shape[1])
    bidirectional=Bidirectional(rnn)
    bidirectional2=Bidirectional(rnn2)
    repeatVector=RepeatVector(output_sequence_length)
    logits = TimeDistributed(Dense(target_vocab_size, activation='softmax'))

    model = Sequential()
    model.add(embedding)
    model.add(bidirectional)
    model.add(repeatVector)
    model.add(rnn2)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    # TODO: Implement
    print(model.summary())
    #plot_model(model, to_file='model.png', show_shapes=True)
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')




## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed input data
    :param y: Preprocessed target data
    :param x_tk: input tokenizer
    :param y_tk: target tokenizer
    """

    # TODO: Train neural network using model_final
    tmp_x = pad(preproc_input_sentences)
    print(tmp_x.shape)
    print(preproc_san_sent.shape[1])

    model= model_final(tmp_x.shape, preproc_san_sent.shape[1], input_vocab_size, target_vocab_size)
    model.fit(tmp_x, preproc_san_sent, batch_size=1024, epochs=10, validation_split=0.2)


    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_input_sentences, preproc_san_sent, input_tokenizer, target_tokenizer)